In [2]:
# Import pandas
import pandas as pd

df = pd.read_json('data.json')

In [2]:
import geopandas as gpd
import osmnx as ox

place_name = "Youssoufia, Morocco"
G = ox.graph_from_place(place_name, network_type='all_private')

gdf = gpd.read_file('map.geojson')

In [11]:
# Convert the graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [34]:
gdf_nodes.drop(gdf_nodes[gdf_nodes['highway'].isnull()].index, inplace=True)

In [52]:
gdf.drop([0, 1], inplace=True)

In [57]:
for x, y in gdf.iterrows():
    print(y.geometry.coords)

In [54]:
import folium

# Créer une carte centrée sur Youssoufia
map_youssoufia = folium.Map(location=[32.23630130013652, -8.51364643783117], zoom_start=13)

tooltip = 'Click to see the picture'

# Ajouter les nœuds sur la carte
for idx, node in gdf_nodes.iterrows():
    folium.Marker([node.geometry.y, node.geometry.x], popup=node['highway']).add_to(map_youssoufia)

# Ajouter des endroits personnalisé
for x, y in gdf.iterrows():
    folium.Marker(location=y.geometry.coords, popup=y['name']).add_to(map_youssoufia)

# Ajouter les arêtes sur la carte
for idx, edge in gdf_edges.iterrows():
    folium.PolyLine(locations=edge.geometry.coords, color='red').add_to(map_youssoufia)

# Afficher la carte
map_youssoufia

ValueError: Expected two (lat, lon) values for location, instead got: <shapely.coords.CoordinateSequence object at 0x7f8b28beba00>.

In [9]:
import folium

# Create a map centered on Youssoufia city in Morocco
map_youssoufia = folium.Map(location=[32.23630130013652, -8.51364643783117], zoom_start=13)

# Add a marker for a specific location (latitude, longitude)
folium.Marker(location=[32.23630130013652, -8.51364643783117], popup='Fatima Zahrae home').add_to(map_youssoufia)

# Display the map in the Jupyter Notebook
map_youssoufia
